In [1]:
from datetime import date, timedelta, datetime
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import yfinance as yf
import base64
import io
import warnings
warnings.filterwarnings('ignore')
from icecream import ic
def time_format():
    return f'{datetime.now()}|> '
ic.configureOutput(includeContext=False)

In [2]:
def get_stock_data(stock, start_date):
    end_date = (date.today() - timedelta(days=1)).strftime('%Y-%m-%d')
    data = yf.download([stock], start=start_date, end=end_date)
    if data.shape[0] == 0:
        return data
    df = pd.date_range(start_date, end_date).to_frame()
    df=df.join(data)
    df['Symbol'] = stock
    df.rename(columns={0:'Date'}, inplace=True)
    return df

In [4]:
# get_stock_data('')


In [5]:
df = get_stock_data('NTPC.NS', '2023-10-09')

[*********************100%%**********************]  1 of 1 completed


In [6]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
2023-10-09,2023-10-09,233.850006,236.300003,230.500000,234.699997,232.455261,10659145.0,NTPC.NS
2023-10-10,2023-10-10,234.899994,236.850006,233.750000,235.899994,233.643784,11665907.0,NTPC.NS
2023-10-11,2023-10-11,236.250000,239.699997,236.000000,238.699997,236.417007,14313043.0,NTPC.NS
2023-10-12,2023-10-12,238.949997,242.750000,238.600006,241.600006,239.289276,13170027.0,NTPC.NS
2023-10-13,2023-10-13,240.550003,243.800003,239.600006,242.050003,239.734970,11880728.0,NTPC.NS
...,...,...,...,...,...,...,...,...
2024-01-22,2024-01-22,308.450012,308.450012,308.450012,308.450012,308.450012,0.0,NTPC.NS
2024-01-23,2024-01-23,313.450012,315.299988,300.299988,302.600006,302.600006,21652362.0,NTPC.NS
2024-01-24,2024-01-24,302.950012,310.700012,298.250000,308.850006,308.850006,23922028.0,NTPC.NS
2024-01-25,2024-01-25,310.000000,317.200012,309.200012,314.750000,314.750000,32525422.0,NTPC.NS


In [7]:
ohlc_dict = {
    'Open':'first',
    'High':'max',
    'Low':'min',
    'Close':'last',
    'Volume':'sum'
    }

dfw = df.resample('W-Fri').agg(ohlc_dict)

In [8]:
vol = dfw.Volume.to_frame()
vol['shift1'] = vol['Volume'].shift(1)
vol['shift2'] = vol['Volume'].shift(2)


vol['case1'] = vol.apply(lambda x: x[0]<=x[1]*1.2 and x[1]<=x[2]*1.2, axis=1)

In [9]:
case1 = vol.loc[vol['case1']]
len(case1)

2

In [10]:
hldf = dfw[['High', 'Low']]
hldf['highshift1'] = hldf['High'].shift(1)
hldf['highshift2'] = hldf['High'].shift(2)
hldf['lowshift1'] = hldf['Low'].shift(1)
hldf['lowshift2'] = hldf['Low'].shift(2)
hldf['case2'] = hldf.apply(lambda x: x['High']<=x['highshift1']*1.02 and x['highshift1']<=x['highshift2']*1.02 and x['Low']>=x['lowshift1']*0.98 and x['lowshift1']>=x['lowshift2']*0.98, axis=1)

In [11]:
case2 = hldf.loc[hldf.case2]

In [12]:
final_case = case1.join(case2, how='inner')

In [273]:
# final_case.index.tolist()

In [13]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=1, row_heights=[0.8,0.2], shared_xaxes=True, vertical_spacing=0)

fig.add_trace(go.Candlestick(x=dfw.index,open=dfw['Open'],high=dfw['High'],low=dfw['Low'],close=dfw['Close'],
                                            name=''), row=1, col=1)

fig.add_trace(go.Bar(x=dfw.index, y=dfw['Volume']), row=2, col=1)

for i in final_case.index.tolist():
    fig.add_shape(
            x0=i, x1=i, y0=0, y1=1, xref='x', yref='paper',
            line_width=0.5)
    
fig.update_xaxes(
    visible=True,
    rangeslider={'visible':False},
    showspikes=False, spikemode='across', spikesnap='cursor',
    showgrid=False, zeroline=True, gridwidth=1, gridcolor='#3E464E', zerolinewidth=3, zerolinecolor='#3E464E'
)
fig.update_layout(
    margin={'l':0, 'b':0, 't':0}
)

fig.show()

In [104]:
df = pd.read_csv('../../assets/EQUITY_L.csv')
df.columns

Index(['SYMBOL', 'NAME OF COMPANY', ' SERIES', ' DATE OF LISTING',
       ' PAID UP VALUE', ' MARKET LOT', ' ISIN NUMBER', ' FACE VALUE'],
      dtype='object')

In [ ]:
df = pd.read_csv('../../assets/EQUITY_L.csv')
fd = df.loc[df[' FACE VALUE'] == 10]
fd['SYMBOL'].tolist()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome('C:\\Users\\Hp\\Documents\\personal\\webdriver\\chromedriver.exe')

In [121]:
from selenium import webdriver
from selenium.webdriver.common.by import By
browser = webdriver.Chrome()

browser.get('https://www.tradingview.com/screener/')


In [149]:
stocks = browser.find_elements(By.CLASS_NAME, 'tv-data-table__tbody')[1].find_elements(By.TAG_NAME, 'tr')

In [ ]:
stock_data = list()
for i in range(len(stocks)):
    stock_symbol = stocks[i].find_element(By.CLASS_NAME, 'tv-screener__symbol').text
    stock_name = stocks[i].find_element(By.CLASS_NAME, 'tv-screener__description').text
    stock_data.append([stock_symbol, stock_name]+[i.text for i in stocks[i].find_elements(By.TAG_NAME, 'td')][1:])


stock_data_label = ['Symbol', 'Name', 'Price', 'chg%', 'chg', 'TR', 'Vol', 'Vol_Price', 'Mkt_Cap', 'P/E', 'EPS(TTM)', 'EMP', 'Sector']
pd.DataFrame(data = stock_data, columns=stock_data_label)

In [204]:
df = pd.DataFrame(data = stock_data, columns=stock_data_label)


In [205]:
mkt = df.Mkt_Cap.str.split('INR', expand=True)[0]

In [206]:
def convert(x):
    if 'T' in x:
        return float(x[:-1])*100_000
    elif 'B' in x:
        return float(x[:-1])*100
    elif 'M' in x:
        return float(x[:-1])*0.1

df['Mkt_Cap'] = mkt.map(convert).astype(float)

In [207]:
df['Price'] = df.Price.str.split('INR', expand=True)[0].str.split('USD', expand=True)[0].astype(float)

In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Symbol     2400 non-null   object 
 1   Name       2400 non-null   object 
 2   Price      2400 non-null   float64
 3   chg%       2400 non-null   object 
 4   chg        2400 non-null   object 
 5   TR         2400 non-null   object 
 6   Vol        2400 non-null   object 
 7   Vol_Price  2400 non-null   object 
 8   Mkt_Cap    2400 non-null   float64
 9   P/E        2400 non-null   object 
 10  EPS(TTM)   2400 non-null   object 
 11  EMP        2400 non-null   object 
 12  Sector     2400 non-null   object 
dtypes: float64(2), object(11)
memory usage: 243.9+ KB


In [214]:
df.to_csv('../../assets/stock_data2.csv', index=False)

In [213]:
df['Symbol'] = df.Symbol.apply(lambda x:x+".NS")

In [14]:
fd = pd.read_csv('../../assets/stock_data2.csv')
fd.sample(3)

FileNotFoundError: [Errno 2] No such file or directory: '../../assets/stock_data2.csv'

In [8]:
df2 = get_stock_data(fd.Symbol[0], "2013-01-01")

[*********************100%%**********************]  1 of 1 completed


In [9]:
df1 = get_stock_data(fd.Symbol[1], "2023-10-09")
df1

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
2023-10-09,2023-10-09,3645.050049,3679.000000,3631.000000,3638.350098,3638.350098,4233074.0,TCS.NS
2023-10-10,2023-10-10,3640.050049,3648.000000,3601.350098,3628.899902,3628.899902,1848148.0,TCS.NS
2023-10-11,2023-10-11,3650.000000,3650.000000,3605.000000,3609.899902,3609.899902,2426417.0,TCS.NS
2023-10-12,2023-10-12,3575.000000,3588.649902,3538.100098,3542.550049,3542.550049,4241281.0,TCS.NS
2023-10-13,2023-10-13,3530.000000,3577.000000,3525.000000,3570.850098,3570.850098,2890357.0,TCS.NS
2023-10-14,2023-10-14,NaN,NaN,NaN,NaN,NaN,NaN,TCS.NS


In [ ]:

stock_having_pro = list()
final_df = pd.DataFrame()
for i in range(0,1000):#
    try:
        df = get_stock_data(fd.Symbol[i], "2023-10-09")
        final_df = pd.concat([final_df, df])
    except:
        stock_having_pro.append(fd.Symbol[i])


In [21]:
# final_df.to_csv('../../assets/stock_historical_data.csv', index=False)
# previous_df = pd.read_csv('../../assets/stock_historical_data.csv')
# previous_df.info()
pd.concat([previous_df, final_df]).to_csv('../../assets/stock_historical_data.csv', index=False)

In [46]:
# historical_data = pd.read_csv('../../assets/stock_historical_data.csv')
# historical_data.head()
historical_data['Date'] = pd.to_datetime(historical_data.Date, format='mixed')
historical_data.set_index('Date', inplace=True)

In [36]:
def get_volume_case(df, factor=1):
    vol = df.Volume.to_frame()
    vol['shift1'] = vol['Volume'].shift(1)
    vol['shift2'] = vol['Volume'].shift(2)
    vol['case1'] = vol.apply(lambda x: x[0]<=x[1]*factor and x[1]<=x[2]*factor, axis=1)
    case1 = vol.loc[vol['case1']]
    return case1

def get_high_low_case(df, factor_bool=False):
    hldf = df[['High', 'Low']]
    factor = 0
    if factor_bool:    
        open = df.Open.tolist()[-1]
        if open>= 1000:
            factor = 0.01
        elif open >= 700:
            factor = 0.02
        elif open >= 400:
            factor = 0.03
        elif open>= 100:
            factor = 0.04
        else:
            factor = 0.05
    high_factor, low_factor = 1 + factor, 1 - factor
    hldf['highshift1'] = hldf['High'].shift(1)
    hldf['highshift2'] = hldf['High'].shift(2)
    hldf['lowshift1'] = hldf['Low'].shift(1)
    hldf['lowshift2'] = hldf['Low'].shift(2)
    hldf
    hldf['case2'] = hldf.apply(lambda x: x['High']<=x['highshift1']*high_factor and x['highshift1']<=x['highshift2']*high_factor and x['Low']>=x['lowshift1']*low_factor and x['lowshift1']>=x['lowshift2']*low_factor, axis=1)
    case2 = hldf.loc[hldf.case2]
    return case2

In [59]:
unique_symbol = historical_data.Symbol.unique().tolist()
current_eligible = {'strict':[], 'some_strict':[]}
problem_stock = list()
for i in range(0, 1000):#len(unique_symbol)
    symbol = unique_symbol[i]
    try:
            
        # ic(symbol)
        df = historical_data.query('Symbol == @symbol')
        df = df.loc[datetime.now()-timedelta(weeks=3):]
        dfw = df.resample('W-Fri').agg(ohlc_dict).iloc[:3]
        # ic(dfw)
        #strict
        case1 = get_volume_case(dfw, 1)
        case2 = get_high_low_case(dfw, False)
        final_df = case1.join(case2, how='inner')
        if final_df.shape[0] != 0:
            current_eligible['strict'].append(symbol)
        
        #strict some
        case1 = get_volume_case(dfw, 1.05)
        case2 = get_high_low_case(dfw, True)
        final_df = case1.join(case2, how='inner')
        if final_df.shape[0] != 0:
            current_eligible['some_strict'].append(symbol)
    except:
        problem_stock.append(symbol)
    

In [60]:
st1 = pd.DataFrame.from_dict(current_eligible['strict'])
st1['case'] = 'strict'
st2 = pd.DataFrame.from_dict([i for i in current_eligible['some_strict'] if i not in current_eligible['strict']])
st2['case'] = 'some_strict'

In [61]:
pd.concat([st1, st2]).to_csv(f'../../assets/{date.today()}.csv', index=False)

In [62]:
current_eligible

{'strict': ['TATAINVEST.NS', 'GREENLAM.NS', 'BARBEQUE.NS'],
 'some_strict': ['ICICIBANK.NS',
  'TITAN.NS',
  'SUNPHARMA.NS',
  'NTPC.NS',
  'GODREJCP.NS',
  'DABUR.NS',
  'PNB.NS',
  'VEDL.NS',
  'RECLTD.NS',
  'CANBK.NS',
  'BERGEPAINT.NS',
  'HAL.NS',
  'ICICIGI.NS',
  'TORNTPHARM.NS',
  'ZYDUSLIFE.NS',
  'PGHH.NS',
  'NHPC.NS',
  'YESBANK.NS',
  'ABCAPITAL.NS',
  'PAGEIND.NS',
  'CENTRALBK.NS',
  'NYKAA.NS',
  'FEDERALBNK.NS',
  'UNOMINDA.NS',
  'DEVYANI.NS',
  'NIACL.NS',
  'GLENMARK.NS',
  'IDFC.NS',
  'HUDCO.NS',
  'TATAINVEST.NS',
  'ASAHIINDIA.NS',
  'RBLBANK.NS',
  'COCHINSHIP.NS',
  'FINPIPE.NS',
  'NUVOCO.NS',
  'ERIS.NS',
  'MANAPPURAM.NS',
  'AEGISCHEM.NS',
  'KARURVYSYA.NS',
  'CANFINHOME.NS',
  'GRSE.NS',
  'KRBL.NS',
  'GRANULES.NS',
  'IONEXCHANG.NS',
  'SPARC.NS',
  'SAREGAMA.NS',
  'NAVA.NS',
  'ALLCARGO.NS',
  'RBA.NS',
  'HNDFDS.NS',
  'BORORENEW.NS',
  'SOUTHBANK.NS',
  'GREENLAM.NS',
  'GREENPANEL.NS',
  'STAR.NS',
  'HCC.NS',
  'IFBIND.NS',
  'UFLEX.NS',
  'JIND

In [57]:
len(problem_stock)

0